<a href="https://colab.research.google.com/github/awesome-astra/docs/blob/main/docs/pages/tools/notebooks/Retrieval_Augmented_Generation_(for_AI_Chatbots).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Demonstration on Relevance with RAG**
Show RAG examples using different similarity distance calculations available in Astra and how search results can be different depending on the question asked.

## **Getting Started with this notebook**

[Create a serverless database with Astra Vector Search and set up its schema.](https://docs.datastax.com/en/astra-serverless/docs/vector-search/chatbot-quickstart.html)


## **Install libraries**

In [ ]:
!pip install openai pandas jupyter-datatables cassandra-driver datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 103.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.8/237.8 kB 32.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 96.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.6 MB/s eta 0:00:00
  Preparing metadata (setup.py)

## **Import Modules**

In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory
from cassandra.query import SimpleStatement
import openai
import numpy
import pandas as pd
import time
from getpass import getpass

## **Keys & Environment Variables**

In [ ]:
# Input your Astra DB token string, the one starting with "AstraCS:..."
ASTRA_DB_TOKEN_BASED_PASSWORD = getpass('Your Astra DB Token ("AstraCS:..."): ')

Your Astra DB Token ("AstraCS:..."): ··········


In [ ]:
# Input your OpenAI api key
openai_api_key = getpass('Enter your OpenAI API key: ')

Enter your OpenAI API key: ··········


In [ ]:
# Input your database keyspace name:
my_ks = input('Your Astra Keyspace name: ')

Your Astra Keyspace name: vector_preview


In [ ]:
# keys and tokens here
openai.api_key = openai_api_key
cass_user = 'token'
cass_pw = ASTRA_DB_TOKEN_BASED_PASSWORD

In [ ]:
# Upload your Secure Connect Bundle zipfile:
import os
from google.colab import files


print('Please upload your Secure Connect Bundle')
uploaded = files.upload()
if uploaded:
    astraBundleFileTitle = list(uploaded.keys())[0]
    scb_path = os.path.join(os.getcwd(), astraBundleFileTitle)
else:
    raise ValueError(
        'Cannot proceed without Secure Connect Bundle. Please re-run the cell.'
    )

Please upload your Secure Connect Bundle


Saving secure-connect-vectordemo.zip to secure-connect-vectordemo.zip


# **Select a model to compute embeddings**

In [ ]:
model_id = "text-embedding-ada-002"

# **Connect to Astra with Vector Search**

In [ ]:
cloud_config= {
  'secure_connect_bundle': scb_path
}
auth_provider = PlainTextAuthProvider(cass_user, cass_pw)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider, protocol_version=4)
session = cluster.connect()
session.set_keyspace(my_ks)
session

# **Drop / Create Schema**

In [ ]:
# only use this to reset the schema
session.execute(f"""DROP INDEX IF EXISTS title_context_desc_cos""")
session.execute(f"""DROP TABLE IF EXISTS squad_cos""")
session.execute(f"""DROP INDEX IF EXISTS title_context_desc_dp""")
session.execute(f"""DROP TABLE IF EXISTS squad_dp""")
session.execute(f"""DROP INDEX IF EXISTS title_context_desc_euc""")
session.execute(f"""DROP TABLE IF EXISTS squad_euc""")

In [ ]:
# # Create Table
session.execute(f"""CREATE TABLE IF NOT EXISTS squad_cos
(id text,
 title text,
 context text,
 question text,
 answers map<text,text>,
 title_context_embedding vector<float, 1536>,
 PRIMARY KEY (id,title))""")

# # Create Index
session.execute("""
    CREATE CUSTOM INDEX IF NOT EXISTS title_context_desc_cos
    ON squad_cos (title_context_embedding)
    USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'
    WITH OPTIONS = {'similarity_function' : 'cosine'}
""")

# # Create Table
session.execute(f"""CREATE TABLE IF NOT EXISTS squad_dp
(id text,
 title text,
 context text,
 question text,
 answers map<text,text>,
 title_context_embedding vector<float, 1536>,
 PRIMARY KEY (id,title))""")

# # Create Index
session.execute("""
    CREATE CUSTOM INDEX IF NOT EXISTS title_context_desc_dp
    ON squad_dp (title_context_embedding)
    USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'
    WITH OPTIONS = {'similarity_function' : 'dot_product'}
""")

# # Create Table
session.execute(f"""CREATE TABLE IF NOT EXISTS squad_euc
(id text,
 title text,
 context text,
 question text,
 answers map<text,text>,
 title_context_embedding vector<float, 1536>,
 PRIMARY KEY (id,title))""")

# # Create Index
session.execute("""
    CREATE CUSTOM INDEX IF NOT EXISTS title_context_desc_euc
    ON squad_euc (title_context_embedding)
    USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'
    WITH OPTIONS = {'similarity_function' : 'euclidean'}
""")

# **Building the Knowledge Base**

We start by constructing our knowledge base. We'll use a mostly prepared dataset called Stanford Question-Answering Dataset (SQuAD) hosted on Hugging Face Datasets. We download it like so:

In [ ]:
from datasets import load_dataset

data = load_dataset('squad', split='train')
data

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

Let's see the 1st 5 records. The dataset does contain duplicate contexts,

In [ ]:
data = data.to_pandas()
data.head()

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'..."
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...


Let's remove the duplicate context and keep the 1st record of the records with same context.

In [ ]:
data.drop_duplicates(subset='context', keep='first', inplace=True)
data.head()

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
5,5733bf84d058e614000b61be,University_of_Notre_Dame,"As at most other universities, Notre Dame's st...",When did the Scholastic Magazine of Notre dame...,"{'text': ['September 1876'], 'answer_start': [..."
10,5733bed24776f41900661188,University_of_Notre_Dame,The university is the major seat of the Congre...,Where is the headquarters of the Congregation ...,"{'text': ['Rome'], 'answer_start': [119]}"
15,5733a6424776f41900660f51,University_of_Notre_Dame,The College of Engineering was established in ...,How many BS level degrees are offered in the C...,"{'text': ['eight'], 'answer_start': [487]}"
20,5733a70c4776f41900660f64,University_of_Notre_Dame,All of Notre Dame's undergraduate students are...,What entity provides help with the management ...,"{'text': ['Learning Resource Center'], 'answer..."


# **Load the table with data and create text embeddings**

* If you are using free trial of the OpenAPI,  the rate limit is about 60/min.  Please adjust the ***counter*** variable below according to your rate limit.
* It takes a long time to load the full data so it is set as 300 here.   Please adjust the ***total*** variable below according to the number of data to test.  Please note that the more data is loaded the longer it takes to load.
* This will takes around **6 minutes** to load the 300 data.

In [ ]:
counter = 0;
total = 0
for id, row in data.iterrows():

  # This is to convert the content of 'answers' to be map<text,text> so it can be saved into Astra.
  converted_answers = dict()
  converted_answers['text'] = row.answers['text'][0]
  converted_answers['answer_start'] = str(row.answers['answer_start'][0])

  # Create Embedding for each conversation row, save them to the database
  full_chunk = f"{row.context} {row.title}"
  embedding = openai.Embedding.create(input=full_chunk, model=model_id)['data'][0]['embedding']

  query_cos = SimpleStatement(
                f"""
                INSERT INTO squad_cos
                (id, title, context, question, answers, title_context_embedding)
                VALUES (%s, %s, %s, %s, %s, %s)
                """
            )

  query_dp = SimpleStatement(
                f"""
                INSERT INTO squad_dp
                (id, title, context, question, answers, title_context_embedding)
                VALUES (%s, %s, %s, %s, %s, %s)
                """
            )

  query_euc = SimpleStatement(
                f"""
                INSERT INTO squad_euc
                (id, title, context, question, answers, title_context_embedding)
                VALUES (%s, %s, %s, %s, %s, %s)
                """
            )

  display(row)

  session.execute(query_cos, (row.id, row.title, row.context, row.question, converted_answers, embedding))
  session.execute(query_dp, (row.id, row.title, row.context, row.question, converted_answers, embedding))
  session.execute(query_euc, (row.id, row.title, row.context, row.question, converted_answers, embedding))

  # With free trial of openAI, the rate limit is set as 60/per min.  Please set this counter depends on your own rate limit.
  counter += 1

  # It takes a long time to load all data.  It is set as 300 so it takes a few mins to load.
  total += 1

  if(total >= 300):
    break

  # With OpenAPI free trial,  the rate limit is 60 / per min.   So when hit 60 requests,  the program will sleep for 60 seconds.
  if (counter >= 300):
    counter = 0;
    time.sleep(60)


id                                   5733be284776f41900661182
title                                University_of_Notre_Dame
context     Architecturally, the school has a Catholic cha...
question    To whom did the Virgin Mary allegedly appear i...
answers     {'text': ['Saint Bernadette Soubirous'], 'answ...
Name: 0, dtype: object

id                                   5733bf84d058e614000b61be
title                                University_of_Notre_Dame
context     As at most other universities, Notre Dame's st...
question    When did the Scholastic Magazine of Notre dame...
answers     {'text': ['September 1876'], 'answer_start': [...
Name: 5, dtype: object

id                                   5733bed24776f41900661188
title                                University_of_Notre_Dame
context     The university is the major seat of the Congre...
question    Where is the headquarters of the Congregation ...
answers             {'text': ['Rome'], 'answer_start': [119]}
Name: 10, dtype: object

id                                   5733a6424776f41900660f51
title                                University_of_Notre_Dame
context     The College of Engineering was established in ...
question    How many BS level degrees are offered in the C...
answers            {'text': ['eight'], 'answer_start': [487]}
Name: 15, dtype: object

id                                   5733a70c4776f41900660f64
title                                University_of_Notre_Dame
context     All of Notre Dame's undergraduate students are...
question    What entity provides help with the management ...
answers     {'text': ['Learning Resource Center'], 'answer...
Name: 20, dtype: object

...

id                                   56cedc8eaab44d1400b88b23
title                                           New_York_City
context     On August 24, 1673, Dutch captain Anthonio Col...
question    What was the regnal name of the Prince of Orange?
answers      {'text': ['William III'], 'answer_start': [164]}
Name: 3885, dtype: object

id                                   56cedd1caab44d1400b88b41
title                                           New_York_City
context     At the end of the Second Anglo-Dutch War, the ...
question    After what war did the English receive New Ams...
answers     {'text': ['Second Anglo-Dutch War'], 'answer_s...
Name: 3892, dtype: object

id                                   56ceddd9aab44d1400b88b5f
title                                           New_York_City
context     New York grew in importance as a trading port ...
question    In 1730, what percentage of New York household...
answers              {'text': ['42%'], 'answer_start': [132]}
Name: 3897, dtype: object

id                                   56cede40aab44d1400b88b71
title                                           New_York_City
context     The trial in Manhattan of John Peter Zenger in...
question            In what year was John Peter Zenger tried?
answers              {'text': ['1735'], 'answer_start': [47]}
Name: 3903, dtype: object

id                                   56cedf11aab44d1400b88b93
title                                           New_York_City
context     The Battle of Long Island, the largest battle ...
question    What was the biggest battle of the American Re...
answers     {'text': ['Battle of Long Island'], 'answer_st...
Name: 3911, dtype: object

# **Convert a query string into a text embedding to use as part of the query**

Provide a question to find out the information of the university that you are interested and see how it works with Vector Search and ChatGPT.

Here we use the same API that we used to calculate embeddings for each row in the database, but this time we are using your input question to calculate a vector to use in a query.

In [ ]:
# Question to find out the information that you need.
customer_input = "when was the college of engineering in the University of Notre Dame established?"

# Create embedding based on same model
embedding = openai.Embedding.create(input=customer_input, model=model_id)['data'][0]['embedding']
display(embedding)

[-0.011592327617108822,
 -0.024977494031190872,
 -0.0011400480289012194,
 -0.027979139238595963,
 -0.002911664079874754,
 -0.01697763241827488,
 0.0024430814664810896,
 -0.006960829719901085,
 -0.032189592719078064,
 0.006128925830125809,
 0.008203592151403427,
 -0.005117058753967285,
 0.03887198865413666,
 -0.00402030348777771,
 -0.006828404031693935,
 -0.0028064027428627014,
 0.0013480241177603602,
 -0.007456576451659203,
 0.007959114387631416,
 0.026933318004012108,
 -0.007605979684740305,
 0.000877743645105511,
 0.006356426049023867,
 -0.0010212046327069402,
 0.0026468129362910986,
 -0.03572094067931175,
 0.027340780943632126,
 -0.022763609886169434,
 0.022600624710321426,
 -0.018947038799524307,
 -0.0021001333370804787,
 -0.016054049134254456,
 -0.03900780901312828,
 -0.018322261050343513,
 -0.030043620616197586,
 -0.012522702105343342,
 -0.009337698109447956,
 0.0001390043762512505,
 0.04403318837285042,
 0.02838660217821598,
 0.037595272064208984,
 0.00550754414871335,
 -0.00481

In [ ]:
# Question to find out the information that you need.
customer_input2 = "Who are some famous John's?"

# Create embedding based on same model
embedding = openai.Embedding.create(input=customer_input2, model=model_id)['data'][0]['embedding']
display(embedding)

[0.0035997226368635893,
 0.009586001746356487,
 -0.013364716432988644,
 -0.01302662119269371,
 -0.03555969148874283,
 0.005336605478078127,
 -0.00750439427793026,
 0.035321034491062164,
 -0.010434554889798164,
 -0.004809574224054813,
 -0.00532666128128767,
 0.011389178223907948,
 -0.015724755823612213,
 0.009811398573219776,
 0.0019473638385534286,
 0.023706460371613503,
 0.023706460371613503,
 0.00820047315210104,
 0.004955419339239597,
 -0.01925155520439148,
 0.0035135415382683277,
 -0.0017352254362776875,
 -0.012966956943273544,
 -0.005207333713769913,
 0.004040572792291641,
 0.028187884017825127,
 0.009983761236071587,
 -0.018973123282194138,
 -0.0061387536115944386,
 -0.022115424275398254,
 0.021386198699474335,
 -0.011488617397844791,
 -0.013165836222469807,
 -0.013682924211025238,
 -0.02228778600692749,
 0.007663498166948557,
 -0.016042962670326233,
 -0.011568170040845871,
 0.021041473373770714,
 0.005850377958267927,
 0.008644637651741505,
 0.006503366399556398,
 0.025138394907

Let's take a look at what a query against a vector index could look like. The query vector has the same dimensions (number of entries in the list) as the embeddings we generated a few steps ago for each row in the database.

In [ ]:
# For a cosine similarity distance calculation, use the embedding to find the information nearest to the question asked.
query_cos = SimpleStatement(
    f"""
    SELECT id, title, answers, context, question, similarity_cosine(title_context_embedding,{embedding}) as similarity
    FROM squad_cos
    ORDER BY title_context_embedding ANN OF {embedding} LIMIT 10;
    """
    )
display(query_cos)

<SimpleStatement query="
    SELECT id, title, answers, context, question, similarity_cosine(title_context_embedding,[0.0035997226368635893, 0.009586001746356487, -0.013364716432988644, -0.01302662119269371, -0.03555969148874283, 0.005336605478078127, -0.00750439427793026, 0.035321034491062164, -0.010434554889798164, -0.004809574224054813, -0.00532666128128767, 0.011389178223907948, -0.015724755823612213, 0.009811398573219776, 0.0019473638385534286, 0.023706460371613503, 0.023706460371613503, 0.00820047315210104, 0.004955419339239597, -0.01925155520439148, 0.0035135415382683277, -0.0017352254362776875, -0.012966956943273544, -0.005207333713769913, 0.004040572792291641, 0.028187884017825127, 0.009983761236071587, -0.018973123282194138, -0.0061387536115944386, -0.022115424275398254, 0.021386198699474335, -0.011488617397844791, -0.013165836222469807, -0.013682924211025238, -0.02228778600692749, 0.007663498166948557, -0.016042962670326233, -0.011568170040845871, 0.021041473373770714, 0.005

In [ ]:
# Do the same thing for a dot product distance calculation, use the embedding to find the information nearest to the question asked.
query_dp = SimpleStatement(
    f"""
    SELECT id, title, answers, context, question, similarity_dot_product(title_context_embedding,{embedding}) as similarity
    FROM squad_dp
    ORDER BY title_context_embedding ANN OF {embedding} LIMIT 10;
    """
    )

display(query_dp)

<SimpleStatement query="
    SELECT id, title, answers, context, question, similarity_dot_product(title_context_embedding,[0.0035997226368635893, 0.009586001746356487, -0.013364716432988644, -0.01302662119269371, -0.03555969148874283, 0.005336605478078127, -0.00750439427793026, 0.035321034491062164, -0.010434554889798164, -0.004809574224054813, -0.00532666128128767, 0.011389178223907948, -0.015724755823612213, 0.009811398573219776, 0.0019473638385534286, 0.023706460371613503, 0.023706460371613503, 0.00820047315210104, 0.004955419339239597, -0.01925155520439148, 0.0035135415382683277, -0.0017352254362776875, -0.012966956943273544, -0.005207333713769913, 0.004040572792291641, 0.028187884017825127, 0.009983761236071587, -0.018973123282194138, -0.0061387536115944386, -0.022115424275398254, 0.021386198699474335, -0.011488617397844791, -0.013165836222469807, -0.013682924211025238, -0.02228778600692749, 0.007663498166948557, -0.016042962670326233, -0.011568170040845871, 0.021041473373770714, 

In [ ]:
# For a cosine similarity distance calculation, use the embedding to find the information nearest to the question asked.
query_euc = SimpleStatement(
    f"""
    SELECT id, title, answers, context, question, similarity_euclidean(title_context_embedding,{embedding}) as similarity
    FROM squad_euc
    ORDER BY title_context_embedding ANN OF {embedding} LIMIT 10;
    """
    )
display(query_euc)

<SimpleStatement query="
    SELECT id, title, answers, context, question, similarity_euclidean(title_context_embedding,[0.0035997226368635893, 0.009586001746356487, -0.013364716432988644, -0.01302662119269371, -0.03555969148874283, 0.005336605478078127, -0.00750439427793026, 0.035321034491062164, -0.010434554889798164, -0.004809574224054813, -0.00532666128128767, 0.011389178223907948, -0.015724755823612213, 0.009811398573219776, 0.0019473638385534286, 0.023706460371613503, 0.023706460371613503, 0.00820047315210104, 0.004955419339239597, -0.01925155520439148, 0.0035135415382683277, -0.0017352254362776875, -0.012966956943273544, -0.005207333713769913, 0.004040572792291641, 0.028187884017825127, 0.009983761236071587, -0.018973123282194138, -0.0061387536115944386, -0.022115424275398254, 0.021386198699474335, -0.011488617397844791, -0.013165836222469807, -0.013682924211025238, -0.02228778600692749, 0.007663498166948557, -0.016042962670326233, -0.011568170040845871, 0.021041473373770714, 0.

# **Find the top 5 results using ANN Similarity**

In [ ]:
results_cos = session.execute(query_cos)
top_5_results_cos = results_cos._current_rows

for row in top_5_results_cos:
  print(f"""{row.similarity} | {row.context}\n""")

0.8841489553451538 | Notre Dame alumni work in various fields. Alumni working in political fields include state governors, members of the United States Congress, and former United States Secretary of State Condoleezza Rice. A notable alumnus of the College of Science is Medicine Nobel Prize winner Eric F. Wieschaus. A number of university heads are alumni, including Notre Dame's current president, the Rev. John Jenkins. Additionally, many alumni are in the media, including talk show hosts Regis Philbin and Phil Donahue, and television and radio personalities such as Mike Golic and Hannah Storm. With the university having high profile sports teams itself, a number of alumni went on to become involved in athletics outside the university, including professional baseball, basketball, football, and ice hockey players, such as Joe Theismann, Joe Montana, Tim Brown, Ross Browner, Rocket Ismail, Ruth Riley, Jeff Samardzija, Jerome Bettis, Brett Lebda, Olympic gold medalist Mariel Zagunis, prof

In [ ]:
results_dp = session.execute(query_dp)
top_5_results_dp = results_dp._current_rows

for row in top_5_results_dp:
  print(f"""{row.similarity} | {row.context}\n""")

0.884148359298706 | Notre Dame alumni work in various fields. Alumni working in political fields include state governors, members of the United States Congress, and former United States Secretary of State Condoleezza Rice. A notable alumnus of the College of Science is Medicine Nobel Prize winner Eric F. Wieschaus. A number of university heads are alumni, including Notre Dame's current president, the Rev. John Jenkins. Additionally, many alumni are in the media, including talk show hosts Regis Philbin and Phil Donahue, and television and radio personalities such as Mike Golic and Hannah Storm. With the university having high profile sports teams itself, a number of alumni went on to become involved in athletics outside the university, including professional baseball, basketball, football, and ice hockey players, such as Joe Theismann, Joe Montana, Tim Brown, Ross Browner, Rocket Ismail, Ruth Riley, Jeff Samardzija, Jerome Bettis, Brett Lebda, Olympic gold medalist Mariel Zagunis, profe

In [ ]:
results_euc = session.execute(query_euc)
top_5_results_euc = results_euc._current_rows

for row in top_5_results_euc:
  print(f"""{row.similarity} | {row.context}\n""")

0.6833369135856628 | Notre Dame alumni work in various fields. Alumni working in political fields include state governors, members of the United States Congress, and former United States Secretary of State Condoleezza Rice. A notable alumnus of the College of Science is Medicine Nobel Prize winner Eric F. Wieschaus. A number of university heads are alumni, including Notre Dame's current president, the Rev. John Jenkins. Additionally, many alumni are in the media, including talk show hosts Regis Philbin and Phil Donahue, and television and radio personalities such as Mike Golic and Hannah Storm. With the university having high profile sports teams itself, a number of alumni went on to become involved in athletics outside the university, including professional baseball, basketball, football, and ice hockey players, such as Joe Theismann, Joe Montana, Tim Brown, Ross Browner, Rocket Ismail, Ruth Riley, Jeff Samardzija, Jerome Bettis, Brett Lebda, Olympic gold medalist Mariel Zagunis, prof